In [63]:
import os
import arcpy
import numpy as np
import math
from datetime import datetime


dirPath = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\White-Goose-Light-Pollution\Data')
inVector = os.path.join(dirPath , 'Vector','points.shp')
biomeVector = os.path.join(dirPath,'Vector','tnc_terr_ecoregions.shp')
inRasterFolder = os.path.join(dirPath,'Images')
outRasterFolder = os.path.join(dirPath,'Images_Extent')

arcpy.env.workspace = inRasterFolder

## getting extent and adding 1000m
desc = arcpy.Describe(inVector)
extent = desc.extent
bufferDistance = 0.0001388  ##1000m in degree units
extentString = str(extent.XMin - bufferDistance) + ' ' + str(extent.YMin - bufferDistance) + ' ' +  \
                str(extent.XMax + bufferDistance) + ' ' + str(extent.YMax + bufferDistance) 
print(extentString)


###clipping each raster
inRasters = arcpy.ListRasters()
for raster in inRasters:
    rasterDesc =  arcpy.Describe(raster)
    rasterName = rasterDesc.name
    outRaster = os.path.join(outRasterFolder, rasterName)
    
    ##delete raster if exits
    if arcpy.Exists(outRaster):
        arcpy.Delete_management(outRaster)
    
    ## clip raster
    arcpy.Clip_management (raster, extentString, outRaster, \
                     maintain_clipping_extent = 'NO_MAINTAIN_EXTENT')

3.6740312 51.1431912 87.6671388 73.5753088


In [64]:
### Identifying breeding biomes
arcpy.MakeFeatureLayer_management(biomeVector, 'biome')
arcpy.SelectLayerByLocation_management('biome', 'INTERSECT', inVector)
arcpy.SelectLayerByAttribute_management('biome', 'SUBSET_SELECTION', " ECO_NAME LIKE '%Tundra%' ")
arcpy.CopyFeatures_management('biome', os.path.join(dirPath, 'tundra_breed.shp'))

<Result 'D:\\M.Sc. Gesopatial Tecnologies\\Courses\\SIW011 - Python in GIS\\Final Project\\White-Goose-Light-Pollution\\Data\\tundra_breed.shp'>

In [6]:
def checkBreedingTime(date):
    for year in range(2006 , 2010):
        breedingStartTime = datetime(year, 5, 24)
        breedingEndTime = datetime(year, 6, 7)
        if date > breedingStartTime and date < breedingEndTime:
            isBreeding = 1
            break
        else:
            isBreeding = 0
    return isBreeding

In [69]:
def distanceKm(lat1, lon1, lat2, lon2):
    p = math.pi/180
    a = 0.5 - math.cos((lat2-lat1)*p)/2 + math.cos(lat1*p) * math.cos(lat2*p) * (1-math.cos((lon2-lon1)*p))/2
    distance = 2* 6371 * math.asin(math.sqrt(a))
    return distance

In [80]:
lat = 5.89583 
lon = 52.92667
lat2 = 5.91883
lon2 = 53.00167

dist = distanceKm(lat,lon,lat2,lon2)
print(dist)

8.680626354373915


In [84]:
###classifying breeding season and adding time and date fields
listFields = [field.name for field in arcpy.ListFields(inVector)]
checkFields = {'isBreed':'SHORT', 'date':'DATE','speed':'DOUBLE'}
for field, fieldType in checkFields.items():
    if field in listFields:
        arcpy.DeleteField_management(inVector, field)
    arcpy.AddField_management(inVector, field, fieldType)

    
fields = ['SHAPE@XY', 'tag_ident', 'timestamp', 'isBreed', 'date', 'speed']

with arcpy.da.UpdateCursor(inVector,fields) as cursor:
    index = 0
    for row in cursor:
        
        time = datetime.strptime(row[2], '%Y-%m-%d %H:%M:%S')
        row[3] = checkBreedingTime(time)
        row[4] = time.date()
        
        lon , lat = row[0]
        currentTime = time
        currentGroup = row[1]
        
        if index > 0:
            if currentGroup == previousGroup:
                distance = distanceKm(lat, lon, lat2, lon2)
                seconds = abs((currentTime - previousTime).total_seconds())
                speed = distance/seconds if seconds > 0 else 0
            else:
                distance = 0
        else:
            distance = 0
        
        row[5] = distance
        
        previousTime = time
        previousGroup = currentGroup
        lon2 = lon
        lat2 = lat
        
        index = index + 1
        cursor.updateRow(row)

In [85]:
inVectorArray = arcpy.da.FeatureClassToNumPyArray(inVector, ('isBreed'))
breedingCount = (inVectorArray['isBreed'] == 1).sum()
nonBreedingCount = (inVectorArray['isBreed'] == 0).sum()
print('Breeding points: ', breedingCount)
print('Non Breeding points: ', nonBreedingCount)

Breeding points:  521
Non Breeding points:  6842
